In [39]:
import glob
import pytesseract
import PIL.Image
from toolz.curried import map, pipe, compose, get
import pandas
from skimage.measure import label, regionprops

ff = lambda *args: compose(*args[::-1])

crop_data = ff(
    PIL.Image.open,
    lambda image: image.crop(box=(0, 960, 1300, 1100))
)

string_data = ff(
    lambda image: pytesseract.image_to_string(image),
    lambda string: string.split(),
    get([1, 3, -1]),
    lambda data: dict(scale=float(data[0].replace('pm', '')),
                      date=data[1],
                      time=data[2])
)

scale_pixels = ff(
    np.array,
    lambda data: data if len(data.shape) ==2 else data[...,0],
    lambda data: label(data, background=0),
    regionprops,
    get(1),
    lambda data: data.bbox[3] - data.bbox[1]
)

make_df = ff(
    glob.glob,
    sorted,
    map(lambda filename: (filename, crop_data(filename))),
    map(lambda data: dict(filename=data[0],
                          scale_pixels=scale_pixels(data[1]),
                          **string_data(data[1]))),
    list,
    pandas.DataFrame
)

data = make_df('data/*.tif')
print(data)

        date                            filename  scale  scale_pixels  \
0  6/17/2015   data/1045_Steel_Nital-etch-01.tif   10.0           107   
1  6/17/2015   data/1045_Steel_Nital-etch-02.tif   10.0           161   
2  6/17/2015   data/1045_Steel_Nital-etch-03.tif   10.0           267   
3  6/17/2015   data/1045_Steel_Nital-etch-04.tif    1.0            80   
4  6/17/2015   data/1045_Steel_Nital-etch-05.tif    1.0            54   
5  6/17/2015   data/1045_Steel_Nital-etch-06.tif   10.0           161   
6  6/17/2015   data/1045_Steel_Nital-etch-07.tif   10.0           267   
7  9/11/2015   data/20150911_1045_Nital_etch.tif   10.0            54   
8  9/11/2015  data/20150911_1045_Nital_etch2.tif   10.0            54   

       time  
0  13:16:51  
1  13:20:17  
2  13:42:26  
3  13:52:11  
4  14:04:46  
5  14:06:17  
6  14:12:37  
7  10:08:55  
8  11:10:14  
